## Sommaire: <a class="anchor" id="sommaire"></a>
* [Sommaire](#sommaire)
* [Package Loading](#package)
* [Model selection](#model_selection)
    * [FD004](#fd004)
    * [FD002](#fd002)
    * [FD003](#fd003)

## Package's loading <a class = "anchor" id="package"> </a>

In [2]:
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import warnings
import tensorflow as tf
import seaborn as sns
import sklearn
import random
import math


from lime.lime_tabular import RecurrentTabularExplainer
from tqdm import tqdm
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.model_selection import GroupKFold
from sklearn import preprocessing
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, GRU
from scipy import optimize
from tensorflow.keras import optimizers


from sp_modif.model_function import *
from sp_modif.methods import *
from sp_modif.data_prep import *
from sp_modif.evaluator import *
from sp_modif.SHAP import *
from sp_modif.L2X import *
from methods import *

%matplotlib inline
warnings.filterwarnings('ignore')

In [3]:
SEED = 0
def set_seed(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(SEED)
    random.seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

# Appeler la fonction pour fixer le seed
set_seed(SEED)

## Model selection <a class = "anchor" id="model_selection"></a>

In [4]:
# 1layers
def model_lstm_1layer(input_shape, nodes_per_layer, dropout, activation, weights_file):
    
    cb = keras.callbacks.EarlyStopping(monitor='loss', patience=4)
    model = Sequential()
    model.add(LSTM(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(256))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  optimizer=Adam(learning_rate=0.001))
    model.save_weights(weights_file)

    return model

In [5]:
# Lower alpha's perform better, so we can ditch a few high ones to reduce the search space
alpha_list = [0.01, 0.05] + list(np.arange(10,60+1,10)/100)

sequence_list = list(np.arange(20,40+1,5))
epoch_list = list(np.arange(5,20+1,5))
nodes_list = [[32], [64], [128], [256]]

# lowest dropout=0.1, because I know zero dropout will yield better training results but worse generalization
dropouts = list(np.arange(1,4)/10)  

# again, earlier testing revealed relu performed significantly worse, so I removed it from the options
activation_functions = ['tanh']
batch_size_list = [32, 64, 128, 256]
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']
sensor_list = [sensor_names]

tuning_options = np.prod([len(alpha_list),
                          len(sequence_list),
                          len(epoch_list),
                          len(nodes_list),
                          len(dropouts),
                          len(activation_functions),
                          len(batch_size_list),
                          len(sensor_list)])
tuning_options

7680

### FD004 <a class="ancho" id="fd004"></a>

In [6]:
# Load data and preprocess
with tf.device('/device:GPU:0'):
    train, test, y_test = prepare_data('FD004.txt')
    print(train.shape, test.shape, y_test.shape)

    remaining_sensors = ["Ps30", "Nf", "phi", "BPR", "farB"] # selection based on R output

    drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

(61249, 27) (41214, 26) (248, 1)


In [7]:
rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

In [8]:
ITERATIONS = 50
SEED = 0

In [9]:
%%time
results = pd.DataFrame()


for i in tqdm.tqdm(range(ITERATIONS)):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    
    # parameter's sample
    weights_file = "weights_file_lstm.h5"
    alpha = random.sample(alpha_list, 1)[0]
    sequence_length = random.sample(sequence_list, 1)[0]
    epochs = 20
    nodes_per_layer = random.sample(nodes_list, 1)[0]
    dropout = random.sample(dropouts, 1)[0]
    activation = random.sample(activation_functions, 1)[0]
    batch_size = random.sample(batch_size_list, 1)[0]
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout,
                             activation, weights_file)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
    
    # Model fitting
    with tf.device('/device:GPU:0'):
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=0)
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse), 'alpha':alpha,
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)

  0%|          | 0/50 [00:00<?, ?it/s]


(52783, 35, 5) (52783, 1) (248, 35, 5)

8/8 [==============================] - 0s 0s/step


  2%|▏         | 1/50 [00:32<26:54, 32.95s/it]

(54028, 30, 5) (54028, 1) (248, 30, 5)
8/8 [==============================] - 0s 2ms/step


  4%|▍         | 2/50 [01:38<41:31, 51.91s/it]

...

(54028, 30, 5) (54028, 1) (248, 30, 5)
8/8 [==============================] - 0s 2ms/step


 98%|█████████▊| 49/50 [3:46:54<07:45, 465.34s/it]

(51538, 40, 5) (51538, 1) (248, 40, 5)
8/8 [==============================] - 0s 7ms/step


100%|██████████| 50/50 [3:52:04<00:00, 278.50s/it]

CPU times: total: 56min 32s
Wall time: 3h 52min 4s


In [10]:
results.sort_values(by="S_score")

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,alpha,nodes,dropout,activation,batch_size,TW
6,17.139551,0.0,1.582473e+03,0.0,293.764221,0.0,0.20,[64],0.3,tanh,32,25
17,19.199793,0.0,1.731451e+03,0.0,368.632080,0.0,0.01,[256],0.3,tanh,128,20
2,16.100150,0.0,1.752538e+03,0.0,259.214813,0.0,0.30,[64],0.2,tanh,32,40
43,15.652385,0.0,1.782797e+03,0.0,244.997147,0.0,0.10,[64],0.2,tanh,32,35
34,16.082235,0.0,1.809801e+03,0.0,258.638306,0.0,0.10,[256],0.3,tanh,32,25
35,18.032118,0.0,1.996625e+03,0.0,325.157288,0.0,0.40,[32],0.2,tanh,32,30
15,17.330257,0.0,2.008192e+03,0.0,300.337830,0.0,0.05,[32],0.3,tanh,32,20
16,17.681289,0.0,2.038687e+03,0.0,312.627960,0.0,0.05,[32],0.1,tanh,32,20
13,18.584638,0.0,2.088771e+03,0.0,345.388733,0.0,0.20,[32],0.3,tanh,32,25
32,19.477533,0.0,2.938860e+03,0.0,379.374268,0.0,0.40,[64],0.2,tanh,32,20


In [11]:
# results.to_csv("results/random_search004.csv")

In [56]:
results.sort_values(by="RMSE")

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW


In [57]:
results.sort_values(by="S_score")

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW


In [58]:
results.sort_values(by="S_score")

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW


### FD002 <a class="anchor" id="fd002"></a>

In [14]:
# Load data and preprocess
with tf.device('/device:GPU:0'):
    train, test, y_test = prepare_data('FD002.txt')
    print(train.shape, test.shape, y_test.shape)
    sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

    remaining_sensors = ["Ps30", "Nf", "phi"] # selection based on main_notebook

    drop_sensors = [element for element in sensor_names if element not in remaining_sensors]
    
rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

ITERATIONS = 50
SEED = 0

(53759, 27) (33991, 26) (259, 1)


In [16]:
%%time
results = pd.DataFrame()

for i in tqdm.tqdm(range(ITERATIONS)):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    
    # parameter's sample
    weights_file = "weights_file_lstm.h5"
    alpha = random.sample(alpha_list, 1)[0]
    sequence_length = random.sample(sequence_list, 1)[0]
    epochs = 20
    nodes_per_layer = random.sample(nodes_list, 1)[0]
    dropout = random.sample(dropouts, 1)[0]
    activation = random.sample(activation_functions, 1)[0]
    batch_size = random.sample(batch_size_list, 1)[0]
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout,
                             activation, weights_file)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
    
    # Model fitting
    with tf.device('/device:GPU:0'):
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=0)
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'alpha' : alpha, 'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)

  0%|          | 0/50 [00:00<?, ?it/s]

(48819, 20, 3) (48819, 1) (259, 20, 3)
9/9 [==============================] - 0s 3ms/step


  2%|▏         | 1/50 [00:44<36:04, 44.16s/it]

(47519, 25, 3) (47519, 1) (259, 25, 3)
9/9 [==============================] - 0s 2ms/step


  4%|▍         | 2/50 [01:36<39:22, 49.22s/it]

...

(47519, 25, 3) (47519, 1) (259, 25, 3)
9/9 [==============================] - 0s 1ms/step


 98%|█████████▊| 49/50 [1:50:50<01:31, 91.46s/it] 

(46219, 30, 3) (46219, 1) (259, 30, 3)
9/9 [==============================] - 1s 4ms/step


100%|██████████| 50/50 [1:51:26<00:00, 133.73s/it]

CPU times: total: 26min 42s
Wall time: 1h 51min 26s


In [17]:
results.sort_values(by='S_score')

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,alpha,nodes,dropout,activation,batch_size,TW
32,16.603718,0.0,1.343367e+03,0.0,275.683502,0.0,0.20,[32],0.1,tanh,128,40
18,16.282424,0.0,1.380667e+03,0.0,265.117310,0.0,0.05,[128],0.2,tanh,64,20
7,19.576382,0.0,1.443432e+03,0.0,383.234741,0.0,0.05,[128],0.2,tanh,256,25
14,17.576449,0.0,1.555623e+03,0.0,308.931549,0.0,0.50,[32],0.2,tanh,32,20
44,17.611794,0.0,1.566269e+03,0.0,310.175293,0.0,0.50,[64],0.3,tanh,64,20
37,17.512567,0.0,1.572190e+03,0.0,306.690002,0.0,0.40,[32],0.2,tanh,64,35
17,17.356768,0.0,1.586986e+03,0.0,301.257355,0.0,0.10,[64],0.2,tanh,64,20
25,17.616426,0.0,1.588816e+03,0.0,310.338440,0.0,0.40,[64],0.2,tanh,256,35
28,18.620923,0.0,1.623487e+03,0.0,346.738739,0.0,0.60,[128],0.1,tanh,256,25
9,17.801028,0.0,1.631151e+03,0.0,316.876587,0.0,0.40,[256],0.2,tanh,32,25


### FD003 <a class="anchor" id="fd003"></a>

In [6]:
# Load data and preprocess
train, test, y_test = prepare_data('FD003.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ["Ps30", "NRc", "W31", "BPR"] # selection based on main_notebook

drop_sensors = [element for element in sensor_names if element not in remaining_sensors]
    
rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

ITERATIONS = 50
SEED = 0

(24720, 27) (16596, 26) (100, 1)


In [7]:
%%time
results003 = pd.DataFrame()

for i in tqdm.tqdm(range(ITERATIONS)):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    
    # parameter's sample
    weights_file = "weights_file_lstm.h5"
    alpha = random.sample(alpha_list, 1)[0]
    sequence_length = random.sample(sequence_list, 1)[0]
    epochs = 20
    nodes_per_layer = random.sample(nodes_list, 1)[0]
    dropout = random.sample(dropouts, 1)[0]
    activation = random.sample(activation_functions, 1)[0]
    batch_size = random.sample(batch_size_list, 1)[0]
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout,
                             activation, weights_file)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
    
    # Model fitting
    with tf.device('/device:GPU:0'):
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=0)
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'alpha' : alpha, 'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results003 = pd.concat([results003, pd.DataFrame(d, index=[0])], ignore_index=True)
    results003.to_csv('results/results003.csv')

  0%|          | 0/50 [00:00<?, ?it/s]


(21320, 35, 4) (21320, 1) (100, 35, 4)

4/4 [==============================] - 0s 0s/step


  2%|▏         | 1/50 [00:13<11:07, 13.62s/it]

(21820, 30, 4) (21820, 1) (100, 30, 4)
4/4 [==============================] - 0s 7ms/step


  4%|▍         | 2/50 [00:40<17:20, 21.67s/it]

...

(21820, 30, 4) (21820, 1) (100, 30, 4)
4/4 [==============================] - 0s 0s/step


 98%|█████████▊| 49/50 [1:16:58<01:38, 98.92s/it] 

(20820, 40, 4) (20820, 1) (100, 40, 4)
4/4 [==============================] - 0s 6ms/step


100%|██████████| 50/50 [1:19:12<00:00, 95.05s/it] 

CPU times: total: 32min 28s
Wall time: 1h 19min 12s


In [9]:
results003.sort_values(by='RMSE')

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,alpha,nodes,dropout,activation,batch_size,TW
43,15.404917,0.0,8.255290e+02,0.0,237.311462,0.0,0.10,[64],0.2,tanh,32,35
27,15.870268,0.0,1.032881e+03,0.0,251.865387,0.0,0.10,[256],0.3,tanh,32,35
2,16.153780,0.0,1.260443e+03,0.0,260.944580,0.0,0.30,[64],0.2,tanh,32,40
35,16.693763,0.0,7.879548e+02,0.0,278.681763,0.0,0.40,[32],0.2,tanh,32,30
30,16.808236,0.0,1.057086e+03,0.0,282.516785,0.0,0.20,[128],0.3,tanh,32,30
16,16.934614,0.0,1.393583e+03,0.0,286.781158,0.0,0.05,[32],0.1,tanh,32,20
4,17.038294,0.0,1.129985e+03,0.0,290.303467,0.0,0.60,[128],0.1,tanh,32,35
15,17.066529,0.0,1.354327e+03,0.0,291.266449,0.0,0.05,[32],0.3,tanh,32,20
6,17.338646,0.0,9.963765e+02,0.0,300.628632,0.0,0.20,[64],0.3,tanh,32,25
29,17.346902,0.0,1.246023e+03,0.0,300.915009,0.0,0.40,[64],0.1,tanh,32,20


### FD001

In [12]:
# Load data and preprocess
train, test, y_test = prepare_data('FD001.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ["Ps30", "Nf", "P30", "BPR"] # selection based on main_notebook

drop_sensors = [element for element in sensor_names if element not in remaining_sensors]
    
rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

ITERATIONS = 50
SEED = 0

(20631, 27) (13096, 26) (100, 1)


In [13]:
%%time
results001 = pd.DataFrame()

for i in tqdm.tqdm(range(ITERATIONS)):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    
    # parameter's sample
    weights_file = "weights_file_lstm.h5"
    alpha = random.sample(alpha_list, 1)[0]
    sequence_length = random.sample(sequence_list, 1)[0]
    epochs = 20
    nodes_per_layer = random.sample(nodes_list, 1)[0]
    dropout = random.sample(dropouts, 1)[0]
    activation = random.sample(activation_functions, 1)[0]
    batch_size = random.sample(batch_size_list, 1)[0]
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout,
                             activation, weights_file)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
    
    # Model fitting
    with tf.device('/device:GPU:0'):
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=0)
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'alpha' : alpha, 'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results001 = pd.concat([results001, pd.DataFrame(d, index=[0])], ignore_index=True)
    results001.to_csv('results/results001.csv')

  0%|          | 0/50 [00:00<?, ?it/s]

(18231, 25, 4) (18231, 1) (100, 25, 4)
4/4 [==============================] - 0s 2ms/step


  2%|▏         | 1/50 [00:20<16:40, 20.41s/it]

(16731, 40, 4) (16731, 1) (100, 40, 4)
4/4 [==============================] - 0s 2ms/step


  4%|▍         | 2/50 [00:34<13:22, 16.71s/it]

...

(17731, 30, 4) (17731, 1) (100, 30, 4)
4/4 [==============================] - 0s 5ms/step


 98%|█████████▊| 49/50 [42:26<00:28, 28.91s/it]

(17731, 30, 4) (17731, 1) (100, 30, 4)
4/4 [==============================] - 0s 0s/step


100%|██████████| 50/50 [42:49<00:00, 51.40s/it]

CPU times: total: 10min 51s
Wall time: 42min 49s


In [14]:
results001.sort_values(by='RMSE')

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,alpha,nodes,dropout,activation,batch_size,TW
21,13.768992,0.0,3.830708e+02,0.0,189.585159,0.0,0.10,[128],0.2,tanh,64,30
22,14.154325,0.0,2.784639e+02,0.0,200.344955,0.0,0.05,[256],0.2,tanh,64,25
46,14.155069,0.0,4.124627e+02,0.0,200.365982,0.0,0.20,[128],0.1,tanh,32,30
28,14.304873,0.0,3.468765e+02,0.0,204.629395,0.0,0.30,[64],0.2,tanh,32,40
3,14.474976,0.0,3.308210e+02,0.0,209.524948,0.0,0.10,[128],0.2,tanh,256,30
44,14.476024,0.0,3.224949e+02,0.0,209.555267,0.0,0.10,[128],0.3,tanh,128,25
32,14.554073,0.0,3.112047e+02,0.0,211.821014,0.0,0.20,[32],0.2,tanh,128,25
33,14.633438,0.0,3.311721e+02,0.0,214.137497,0.0,0.20,[32],0.2,tanh,128,40
6,14.771418,0.0,3.643189e+02,0.0,218.194748,0.0,0.05,[128],0.2,tanh,256,25
12,14.917784,0.0,3.237964e+02,0.0,222.540298,0.0,0.20,[32],0.3,tanh,128,25
